## Pip installs

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install pysam

In [ ]:
!pip install HTSeq

In [5]:
!pip install enformer-pytorch>=0.5

In [3]:
!pip install polars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 30.8 MB/s eta 0:00:0000:0100:01


In [87]:
!pip install scikit-learn

## Loading DNA sequence data from BAMs to create dataset for fine tuning enformer

In [1]:
import torch
import polars as pl
from enformer_pytorch import Enformer, GenomeIntervalDataset
from datasets import concatenate_datasets, load_dataset
import os

/home/groups/cgawad/conda_for_pf_notebook/miniconda3_for_pf/envs/pf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### define variables
hg38_bed_path = "/scratch/users/sschulz/pta_on_normal/Homo_sapiens_assembly38_n25chr.bed"
bam_dir= "/scratch/users/sschulz/pta_on_normal"
fasta_dir = bam_dir + '/fastas'
dataset_name='test_dataset.hf'
preprocess_bam=True
vcf_file=bam_dir +'/CARTPt04_Scan2_svc_merged_extract_snp.vcf.gz'
ref_fasta=bam_dir + '/Homo_sapiens_assembly38.fasta'
results_dir=bam_dir
output_fasta_fn='chr_test_fasta.fasta'
sbatch=0

## Function definitions

In [3]:
def preProcessVCF(vcf_path, ref_fasta, results_dir, output_fasta_fn, sbatch):
    '''
        take directory of a combined vcf file with somatic mutations called for normal cells
        with pta run on them and make a fasta file 
        
        TO-DO: make something that generates the vcf for you if given normal bam files
    '''

    !sh vcf2fasta.sh --vcf_path $vcf_path --ref_fasta $ref_fasta --results_dir $results_dir --output_name $output_fasta_fn --sbatch $sbatch

In [4]:
def makeEnformerDataset(fasta_path, bed_path):
    '''
        makes enformer dataset for genome intervals in bed file at bed_path, returns the dataset
    '''
    #filter_train = lambda df: df.filter(pl.col('column_4') == 'train')
    ds = GenomeIntervalDataset(
        bed_file = bed_path, ##<- this can just be the whole hg38.bed for all chromosomes
        fasta_file = fasta_path,  ## path to fasta file
        #filter_df_fn = filter_train,                        # filter dataframe function
        return_seq_indices = False,                          # return nucleotide indices (ACGTN) or one hot encodings
        shift_augs = (-2, 2),                               # random shift augmentations from -2 to +2 basepairs
        rc_aug = True,                                      # use reverse complement augmentation with 50% probability
        context_length = 196_608,
        return_augs = True                                  # return the augmentation meta data
        )
    return ds

In [5]:
def trainEnformerPTA(dataset):
    '''
        should train the enformer model
    '''
    
    #filter_train = lambda df: df.filter(pl.col('column_4') == 'train')

    model = HeadAdapterWrapper(
                enformer = dataset,
                num_tracks = 128,
                post_transformer_embed = False   # by default, embeddings are taken from after the final pointwise block w/ conv -> gelu - but if you'd like the embeddings right after the transformer block with a learned layernorm, set this to True
            ).cuda()

    return model

In [13]:
## this was all taken from lucidrains/enformer-pytorch, i have no clue if i could just source this somehow and i don't care

def torch_fromstring(seq_strs):
    def cast_list(t):
        return t if isinstance(t, list) else [t]
    batched = not isinstance(seq_strs, str)
    seq_strs = cast_list(seq_strs)
    np_seq_chrs = list(map(lambda t: np.fromstring(t, dtype = np.uint8), seq_strs))
    seq_chrs = list(map(torch.from_numpy, np_seq_chrs))
    return torch.stack(seq_chrs) if batched else seq_chrs[0]

def str_to_one_hot(seq_strs):
    seq_chrs = torch_fromstring(seq_strs)
    return one_hot_embed[seq_chrs.long()]

one_hot_embed = torch.zeros(256, 4)
one_hot_embed[ord('a')] = torch.Tensor([1., 0., 0., 0.])
one_hot_embed[ord('c')] = torch.Tensor([0., 1., 0., 0.])
one_hot_embed[ord('g')] = torch.Tensor([0., 0., 1., 0.])
one_hot_embed[ord('t')] = torch.Tensor([0., 0., 0., 1.])
one_hot_embed[ord('n')] = torch.Tensor([0., 0., 0., 0.])
one_hot_embed[ord('A')] = torch.Tensor([1., 0., 0., 0.])
one_hot_embed[ord('C')] = torch.Tensor([0., 1., 0., 0.])
one_hot_embed[ord('G')] = torch.Tensor([0., 0., 1., 0.])
one_hot_embed[ord('T')] = torch.Tensor([0., 0., 0., 1.])
one_hot_embed[ord('N')] = torch.Tensor([0., 0., 0., 0.])
one_hot_embed[ord('.')] = torch.Tensor([0.25, 0.25, 0.25, 0.25])

## Preprocessing

In [33]:
vcf_file=bam_dir +'/CARTPt04_Scan2_svc_merged_extract_snp.vcf.gz'
ref_fasta=bam_dir + '/Homo_sapiens_assembly38.fasta'
results_dir=bam_dir
output_fasta_fn='test_fasta.fasta'
if preprocess_bam:
    preProcessVCF(vcf_file, ref_fasta, results_dir, output_fasta_fn, sbatch)

Note: the --sample option not given, applying all records regardless of the genotype
The site chr1:159782470 overlaps with another variant, skipping...
The site chr1:206939504 overlaps with another variant, skipping...
The site chr2:32811257 overlaps with another variant, skipping...
The site chr5:141375502 overlaps with another variant, skipping...
The site chr6:349203 overlaps with another variant, skipping...
The site chr6:42236345 overlaps with another variant, skipping...
The site chr7:142760369 overlaps with another variant, skipping...
The site chr7:142772094 overlaps with another variant, skipping...
The site chr9:33796768 overlaps with another variant, skipping...
The site chr9:33796801 overlaps with another variant, skipping...
The site chr9:106926826 overlaps with another variant, skipping...
The site chr11:1017451 overlaps with another variant, skipping...
The site chr11:1017461 overlaps with another variant, skipping...
The site chr11:1017466 overlaps with another variant,

## Make the dataset

In [74]:
fasta_path = bam_dir + '/' + output_fasta_fn

ds = makeEnformerDataset(fasta_path, hg38_bed_path)

In [75]:
ds.df

column_1,column_2,column_3
str,i64,i64
"""chr1""",0,248956422
"""chr2""",0,242193529
"""chr3""",0,198295559
"""chr4""",0,190214555
"""chr5""",0,181538259
"""chr6""",0,170805979
"""chr7""",0,159345973
"""chr8""",0,145138636
"""chr9""",0,138394717


In [77]:
dir(ds)

['__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_protocol',
 'chr_bed_to_fasta_map',
 'df',
 'fasta',
 'return_augs']

In [86]:
len(torch.randint(0, 5, (1, 196_608 // 2,))[0])

98304

## get one-hot encodings with torch

In [9]:
import numpy as np

In [11]:
fastaFilePath = bam_dir + '/final_fasta.fasta'

## first check wtf this looks like in python
# with open(fastaFilePath, 'r') as file:
#     data = file.read()

### put in how long of a sequence u wanna try to pretrain on, 
lines_of_fasta_file=3000
with open(fastaFilePath, 'r') as input_file:
    try:
        head = [next(input_file).strip('\n') for _ in range(length_of_sequence)]
    except StopIteration as e:
        print("Exceeded length of input file, stopping.")
    print(head)


['...']


In [14]:
seq = str_to_one_hot(head)

/tmp/ipykernel_68650/318148619.py:8: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_seq_chrs = list(map(lambda t: np.fromstring(t, dtype = np.uint8), seq_strs))


In [15]:
len(seq)

3000

## Run the model

In [26]:
import torch
from enformer_pytorch import Enformer
from enformer_pytorch.finetune import HeadAdapterWrapper

enformer = Enformer.from_pretrained('EleutherAI/enformer-official-rough')

model = HeadAdapterWrapper(
    enformer = enformer,
    num_tracks = 128,
    post_transformer_embed = False   # by default, embeddings are taken from after the final pointwise block w/ conv -> gelu - but if you'd like the embeddings right after the transformer block with a learned layernorm, set this to True
).cuda()

target = torch.randn(1, 200, 128).cuda()  # 128 tracks

loss = model(seq, target = target)
loss.backward()

OSError: [Errno 28] No space left on device

In [20]:
pred = model(seq, head = 'human') # (3000, 3000)

In [21]:
pred

tensor([[[0.7276, 0.7898, 0.7651,  ..., 0.8316, 0.6573, 0.5948]],

        [[0.7207, 0.7824, 0.8117,  ..., 0.7090, 0.5889, 0.5795]],

        [[0.5686, 0.8286, 0.8646,  ..., 0.7037, 0.6471, 0.6555]],

        ...,

        [[0.7175, 0.7805, 0.7743,  ..., 0.7038, 0.8123, 0.5317]],

        [[0.6933, 0.6910, 0.7489,  ..., 0.7125, 0.6352, 0.6702]],

        [[0.6963, 0.7716, 0.8335,  ..., 0.6973, 0.6662, 0.6498]]],
       grad_fn=<SoftplusBackward0>)

## Old stuff

In [ ]:
dataset_list = []
fai_dir=fasta_dir + '/fais'
!mkdir -p {fai_dir}
for fasta_name in os.listdir(fasta_dir):
    !mv {fasta_dir}/*.fai {fai_dir}
    fasta_path=fasta_dir + '/' + fasta_name
    if not os.path.splitext(fasta_path)[1] == 'fai':
        print('Now collecting dataset for: ' + fasta_path)
        dataset_list.append(makeEnformerDataset(fasta_path, hg38_bed_path))

mv: cannot stat ‘/scratch/users/sschulz/pta_on_normal/fastas/*.fai’: No such file or directory
Now collecting dataset for: /scratch/users/sschulz/pta_on_normal/fastas/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-A9_S57.realigned_deduped_sorted.fasta
Now collecting dataset for: /scratch/users/sschulz/pta_on_normal/fastas/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B10_S65.realigned_deduped_sorted.fasta


In [8]:
def combineDatasets(dataset_list):
    '''
        uses hf datasets to concatenate a list of datasets
    '''
    return concatenate_datasets(dataset_list)

In [ ]:
combined_dataset = combineDatasets(dataset_list)

In [ ]:
combined_dataset.save_to_disk(bam_dir + '/' + dataset_name)

In [ ]:
model = trainEnformerPTA(combined_datset)

In [ ]:
seq = ### enter a sequence to test
pred = model(seq, head = 'human') # (896, 5313) ###<- what does human mean here?